In [1]:
import numpy as np
import pandas as pd

dev_df = pd.read_csv("/Users/sunying/Desktop/champs-scalar-coupling/train.csv")
test_df = pd.read_csv("/Users/sunying/Desktop/champs-scalar-coupling/test.csv")

dev_df.shape, test_df.shape


((4659076, 6), (2505190, 5))

In [2]:
dev_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [3]:
dev_df["median_error"] = np.abs(dev_df["scalar_coupling_constant"] - dev_df.groupby("type")["scalar_coupling_constant"].transform("median"))

stats = dev_df.groupby("type").agg({"id": "count", "median_error": "mean"}).reset_index()
stats["effect"] = stats["id"]*stats["median_error"]
stats["effect"] /= stats["effect"].min()
stats["sw"] = 1/stats["effect"]

dev_df = dev_df.merge(stats[["type", "sw"]], on="type", how="left")

del stats

In [4]:
from sklearn.preprocessing import LabelEncoder

le_type = LabelEncoder()
dev_df["le_type"] = le_type.fit_transform(dev_df["type"].values)
test_df["le_type"] = le_type.transform(test_df["type"].values)

In [5]:
structures = pd.read_csv("/Users/sunying/Desktop/champs-scalar-coupling/structures.csv")
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [6]:
def get_loc(df):
    df = df.merge(structures.rename(columns={"x": "x0", "y": "y0", "z":"z0", "atom": "atom0", "atom_index": "atom_index_0"}), 
                          on=["molecule_name", "atom_index_0"], how="left")
    df = df.merge(structures.rename(columns={"x": "x1", "y": "y1", "z":"z1", "atom": "atom1", "atom_index": "atom_index_1"}), 
                          on=["molecule_name", "atom_index_1"], how="left")
    return df

dev_df = get_loc(dev_df)
test_df = get_loc(test_df)
dev_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,median_error,sw,le_type,atom0,x0,y0,z0,atom1,x1,y1,z1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,3.3984,0.018978,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.0710,0.165878,3,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.0732,0.165878,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.0737,0.165878,3,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,3.3986,0.018978,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [7]:
mol_df = structures.groupby("molecule_name")["atom"].count().reset_index().rename(columns={"atom": "atom_count"})

def get_features(df):
    df = df.merge(mol_df, on="molecule_name", how="left")
    df["dist"] = 1.0/np.linalg.norm(df[["x0", "y0", "z0"]].values - df[["x1", "y1", "z1"]].values, axis=1)
    return df

dev_df = get_features(dev_df)
test_df = get_features(test_df)
dev_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,median_error,sw,le_type,atom0,x0,y0,z0,atom1,x1,y1,z1,atom_count,dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,3.3984,0.018978,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,5,0.915790
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.0710,0.165878,3,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,5,0.560815
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.0732,0.165878,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,5,0.560806
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.0737,0.165878,3,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,5,0.560803
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,3.3986,0.018978,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,5,0.915791


In [8]:
structures["mol_features"] = structures[["x", "y", "z"]].values.tolist()
mol_df = structures.groupby("molecule_name").agg({"mol_features": list, "atom": list}).reset_index()

dev_df = dev_df.merge(mol_df, on="molecule_name", how="left")
test_df = test_df.merge(mol_df, on="molecule_name", how="left")

dev_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,median_error,sw,le_type,atom0,...,y0,z0,atom1,x1,y1,z1,atom_count,dist,mol_features,atom
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,3.3984,0.018978,0,H,...,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,5,0.915790,"[[-0.0126981359, 1.0858041578, 0.0080009957999...","[C, H, H, H, H]"
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.0710,0.165878,3,H,...,-0.006031,0.001976,H,1.011731,1.463751,0.000277,5,0.560815,"[[-0.0126981359, 1.0858041578, 0.0080009957999...","[C, H, H, H, H]"
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.0732,0.165878,3,H,...,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,5,0.560806,"[[-0.0126981359, 1.0858041578, 0.0080009957999...","[C, H, H, H, H]"
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.0737,0.165878,3,H,...,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,5,0.560803,"[[-0.0126981359, 1.0858041578, 0.0080009957999...","[C, H, H, H, H]"
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,3.3986,0.018978,0,H,...,1.463751,0.000277,C,-0.012698,1.085804,0.008001,5,0.915791,"[[-0.0126981359, 1.0858041578, 0.0080009957999...","[C, H, H, H, H]"


In [9]:
for col in dev_df.columns:
    if dev_df[col].dtype == np.int64:
        dev_df[col] = dev_df[col].astype(np.int32)
        
    if dev_df[col].dtype == np.float64:
        dev_df[col] = dev_df[col].astype(np.float32)
        
for col in test_df.columns:
    if test_df[col].dtype == np.int64:
        test_df[col] = test_df[col].astype(np.int32)
        
    if test_df[col].dtype == np.float64:
        test_df[col] = test_df[col].astype(np.float32)

In [10]:
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics.pairwise import euclidean_distances

def get_mol(df):
    M = np.zeros((df.shape[0], 30, 5), dtype=np.float32)

    index = 0

    for properties, cx, mol_features, atom in tqdm_notebook(list(zip(df[["atom_index_0", "atom_index_1"]].values, 
                                                          df[["x0", "y0", "z0", "x1", "y1", "z1"]].values, 
                                                          df["mol_features"].values, df["atom"].values))):
        arr = np.array(mol_features)
        arr = np.delete(arr, np.array([properties[0], properties[1]]), 0)

        atoms = np.delete(np.array(atom), np.array([properties[0], properties[1]]), 0)

        v0 = arr - np.array([cx[0], cx[1], cx[2]])
        v1 = arr - np.array([cx[3], cx[4], cx[5]])

        dist0 = np.sqrt(np.power(v0, 2).sum(axis=1))
        dist1 = np.sqrt(np.power(v1, 2).sum(axis=1))

        M[index, :arr.shape[0], 0] = 1.0/dist0
        M[index, :arr.shape[0], 1] = 1.0/dist1
        M[index, :arr.shape[0], 2] = (atoms == "H")*1.0
        M[index, :arr.shape[0], 3] = (atoms != "H")*1.0
        M[index, :arr.shape[0], 4] = np.sum(v0*v1, axis=1)/(dist0*dist1)
        index += 1
        
    return M

M_train = get_mol(dev_df)
M_test = get_mol(test_df)

/Users/sunying/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [13]:
from keras.layers import *
from keras.models import Model
from keras.optimizers import Nadam

features = ["dist", "atom_count"]

def nn_block(input_layer, size, dropout_rate, activation):
    out_layer = Dense(size, activation=None)(input_layer)
    out_layer = BatchNormalization()(out_layer)
    out_layer = Activation(activation)(out_layer)
    out_layer = Dropout(dropout_rate)(out_layer)
    return out_layer

def cnn_block(input_layer, size, dropout_rate, activation):
    out_layer = Conv1D(size, 1, activation=None)(input_layer)
    out_layer = BatchNormalization()(out_layer)
    out_layer = Activation(activation)(out_layer)
    out_layer = Dropout(dropout_rate)(out_layer)
    return out_layer


def get_model():
    dense_input = Input(shape=(len(features) + 2,))

    type_input = Input(shape=(1,))
    type_emb = Flatten()(Embedding(dev_df["le_type"].max() + 1, 5)(type_input))
    
    mol_input = Input(shape=(M_train.shape[1], M_train.shape[2]))
    mol_layer = cnn_block(mol_input, 200, 0.0, "relu")
    mol_layer = cnn_block(mol_layer, 100, 0.0, "relu")
    
    merged_input = BatchNormalization()(concatenate([dense_input, type_emb, 
                                                     GlobalMaxPooling1D()(mol_layer), GlobalAvgPool1D()(mol_layer)]))
    
    x1 = Dense(200, activation="relu")(merged_input)
    x2 = nn_block(x1, 20, 0.0, "sigmoid")
    
    mol_layer = concatenate([RepeatVector(30)(x2), mol_layer])
    mol_layer = cnn_block(mol_layer, 200, 0.0, "relu")
    mol_layer = cnn_block(mol_layer, 100, 0.0, "relu")
    merged_input = BatchNormalization()(concatenate([dense_input, type_emb, 
                                                     GlobalMaxPooling1D()(mol_layer), GlobalAvgPool1D()(mol_layer)]))
    hidden_layer = concatenate([Dense(600, activation="relu")(merged_input), x1])
    
    hidden_layer = nn_block(hidden_layer, 400, 0.0, "relu")
    hidden_layer = nn_block(hidden_layer, 200, 0.0, "relu")
    hidden_layer = nn_block(hidden_layer, 100, 0.0, "relu")
    
    out = Dense(1, activation="linear")(hidden_layer)
    
    model = Model(inputs=[dense_input, type_input, mol_input], outputs=out)
    return model

Using TensorFlow backend.


In [14]:
import gc

TARGET = "scalar_coupling_constant"
keep_list = [TARGET, "id", "molecule_name", "le_type", "sw", "type"] + features
dev_df.drop([col for col in dev_df.columns if col not in keep_list], 
            axis=1, inplace=True)
test_df.drop([col for col in test_df.columns if col not in keep_list], 
             axis=1, inplace=True)

del mol_df, structures

gc.collect()

28

In [15]:
from sklearn.model_selection import GroupKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

NUM_FOLDS = 4
BATCH_SIZE = 2**10
best_model_path = "best_model.h5"

def to_emb(s):
    return s.values.reshape(-1, 1)

def get_features(df):
    return np.concatenate([df[features].values, np.log(df[features].values)], axis=1)

kfold = GroupKFold(NUM_FOLDS)
oof_preds = np.zeros(dev_df.shape[0])
test_preds = np.zeros(test_df.shape[0])

for train_index, val_index in kfold.split(dev_df[TARGET].values, dev_df[TARGET].values, dev_df["molecule_name"].values):
    
    train_df = dev_df[features + ["sw", "le_type", TARGET]].iloc[train_index]
    val_df = dev_df[features + ["sw", "le_type", TARGET]].iloc[val_index]
    
    train_df["sw"] *= train_df.shape[0]/train_df["sw"].sum()
    val_df["sw"] *= val_df.shape[0]/val_df["sw"].sum()
    
    model = get_model()
    #early_stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=1)
    model_checkpoint = ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=True, verbose=1)
    
    for epochs, lr in zip([3, 3, 3, 3, 6], [0.004, 0.004, 0.001, 0.001, 0.0004]):
        model.compile(loss="mean_absolute_error", optimizer=Nadam(lr=lr))
        model.fit([get_features(train_df), to_emb(train_df["le_type"]), M_train[train_index]], train_df[TARGET], 
                  sample_weight=train_df["sw"].values,
                  validation_data=([get_features(val_df), to_emb(val_df["le_type"]), M_train[val_index]], val_df[TARGET], val_df["sw"].values),
                  verbose=1, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[model_checkpoint])
    
    model.load_weights(best_model_path)
    
    oof_preds[val_index] = model.predict([get_features(val_df), to_emb(val_df["le_type"]), M_train[val_index]], 
                                         verbose=1, batch_size=BATCH_SIZE).ravel()
    
    test_preds += model.predict([get_features(test_df), to_emb(test_df["le_type"]), M_test], 
                                         verbose=1, batch_size=BATCH_SIZE).ravel()/NUM_FOLDS

/Users/sunying/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3494307 samples, validate on 1164769 samples
Epoch 1/3
3494307/3494307 [==============================] - 1447s 414us/step - loss: 0.9307 - val_loss: 1.8178

Epoch 00001: val_loss improved from inf to 1.81785, saving model to best_model.h5
Epoch 2/3
3494307/3494307 [==============================] - 1435s 411us/step - loss: 0.5393 - val_loss: 1.1285

Epoch 00002: val_loss improved from 1.81785 to 1.12845, saving model to best_model.h5
Epoch 3/3
3494307/3494307 [==============================] - 1433s 410us/step - loss: 0.4384 - val_loss: 1.0104

Epoch 00003: val_loss improved from 1.12845 to 1.01043, saving model to best_model.h5
Train on 3494307 samples, validate on 1164769 samples
Epoch 1/3
 186368/3494307 [>.............................] - ETA: 24:29 - loss: 0.4451

KeyboardInterrupt: 

In [16]:
del M_train, M_test

gc.collect()

179

In [17]:
dev_df["oof_pred"] = oof_preds

def competition_metric(df, pred_col):
    df["error"] = np.abs(df[TARGET] - df[pred_col])
    return np.log(df.groupby("type")["error"].mean()).mean()

competition_metric(dev_df, "oof_pred")

1.9974921863036217

In [19]:
test_df[TARGET] = test_preds
test_df.to_csv("/Users/sunying/Desktop/champs-scalar-coupling/submission.csv", columns=["id", TARGET], index=False)

In [20]:
test_df.head()

,id,molecule_name,type,le_type,atom_count,dist,scalar_coupling_constant
0,4659076,dsgdb9nsd_000004,2JHC,2,4,0.442247,0.0
1,4659077,dsgdb9nsd_000004,1JHC,0,4,0.941532,0.0
2,4659078,dsgdb9nsd_000004,3JHH,6,4,0.300908,0.0
3,4659079,dsgdb9nsd_000004,1JHC,0,4,0.941532,0.0
4,4659080,dsgdb9nsd_000004,2JHC,2,4,0.442247,0.0
